## Webscraping Unibet football odds for UEFA2023 U21
Import packages

In [52]:
import datetime as dt
import pandas as pd
import time

import re
import ast
import time
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

Choose the url to be scraped

In [53]:
unibet = 'https://www.unibet.com/betting/sports/filter/football/uefa_championship_u21/all/matches'

# path = '/Users/Wille/Desktop/Wille/Koodi/Chromedriver'

Open the website

In [54]:
driver = webdriver.Chrome()

driver.get(unibet)

Accept cookies to get rid of the popup and scroll down the website to get all the data

In [55]:
accept = driver.find_element(By.XPATH, '//*[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')

time.sleep(3)

accept.click()

time.sleep(1)

scroll_pixels = 500
scroll_iterations = 5

for i in range(scroll_iterations):
    driver.execute_script(f"window.scrollBy(0, {scroll_pixels});")
    driver.implicitly_wait(2)


Initialize storage

In [56]:
teams =[]
x12 = []
odds =[]
teamsOdds =[]
teamsOddsStr = []

Choose the box that we are interested in

In [57]:
box = driver.find_element(By.XPATH, '//*[@id="rightPanel"]/div[3]/div/div[2]/div[1]/div[1]/div/div/div/div[2]/div[1]/div')

# print(box.text)

Find teams in this box

In [58]:
events = box.find_elements(By.CSS_SELECTOR, '[data-test-name="matchEvent"]')    # List of matches


for match in events:
    teams = []
    oddslist = []
    for team in match.find_elements(By.CSS_SELECTOR, '[data-test-name="eventParticipant"]'):    # Iterate through teams in a match
        name = team.find_element(By.CSS_SELECTOR, '[data-test-name="teamName"]')    # Find the name of the team
        teams.append(team.text)     # Make a 2-element list of the opposing teams
    odds = match.find_elements(By.CSS_SELECTOR, '[data-test-name="odds"]')  # Find the odds for the two playing teams
    for odd in odds:
        oddslist.append(odd.text)   # Make a list of the odds like [Odds_team1, Odds_tie, Odds_team2]
    odds_dict_str = {'Teams': f"{teams[0]} - {teams[1]}", 'Odds 1x2': f"|{oddslist[0]}|{oddslist[1]}|{oddslist[2]}|"}
    odds_dict = {'Teams': teams, 'Odds 1x2': oddslist}
    teamsOdds.append(odds_dict)     # Create a dictionary containing the teams and odds
    teamsOddsStr.append(odds_dict_str)  # A string-format dictionary for the pandas table


print(teamsOdds)

pandadict = {'Teams': [], 'Odds 1x2': []}

for match in teamsOddsStr:
    pandadict['Teams'].append(match['Teams'])
    pandadict['Odds 1x2'].append(match['Odds 1x2'])

pandatable = pd.DataFrame.from_dict(pandadict)

print(pandatable)

driver.quit()

[{'Teams': ['Portugal U21', 'Netherlands U21'], 'Odds 1x2': ['2.75', '3.10', '2.60']}, {'Teams': ['Romania U21', 'Ukraine U21'], 'Odds 1x2': ['3.90', '3.70', '1.85']}, {'Teams': ['Georgia U21', 'Belgium U21'], 'Odds 1x2': ['5.20', '4.20', '1.55']}, {'Teams': ['Spain U21', 'Croatia U21'], 'Odds 1x2': ['1.48', '4.30', '6.40']}, {'Teams': ['Switzerland U21', 'Italy U21'], 'Odds 1x2': ['3.75', '3.30', '1.95']}, {'Teams': ['Czech Republic U21', 'Germany U21'], 'Odds 1x2': ['5.50', '3.85', '1.60']}, {'Teams': ['England U21', 'Israel U21'], 'Odds 1x2': ['1.33', '5.00', '8.00']}, {'Teams': ['Norway U21', 'France U21'], 'Odds 1x2': ['7.50', '4.25', '1.40']}]
                              Teams          Odds 1x2
0    Portugal U21 - Netherlands U21  |2.75|3.10|2.60|
1         Romania U21 - Ukraine U21  |3.90|3.70|1.85|
2         Georgia U21 - Belgium U21  |5.20|4.20|1.55|
3           Spain U21 - Croatia U21  |1.48|4.30|6.40|
4       Switzerland U21 - Italy U21  |3.75|3.30|1.95|
5  Czech Republic 